# Projet Knowledge Extraction - Partie A : Preprocessing et Représentation Text

**Université Paris Cité - Master 2 VMI**
**Cours :** IFLCE085 Recherche et extraction sémantique à partir de texte (Prof. Salima Benbernou)

**Équipe :**
- **Partie A (Preprocessing) : Jacques Gastebois**
- Partie B : Boutayna EL MOUJAOUID
- Partie C : Franz Dervis
- Partie D : Aya Benkabour

---

## Étape 1 : Setup et Importations
Objectif : Configurer l'environnement et importer les librairies nécessaires.

In [1]:
import sys
# Installation des dépendances de base
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install spacy

# Téléchargement du modèle spaCy anglais
!{sys.executable} -m spacy download en_core_web_sm



import os
import json
import re
import pickle
import pandas as pd
import numpy as np
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz

# Téléchargement des ressources NLTK (version mise à jour)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
nltk.download('stopwords', quiet=True)

# Chargement du modèle spaCy
nlp = spacy.load('en_core_web_sm')

# Configuration de l'affichage pandas
pd.set_option('display.max_colwidth', 100)

print("Environnement configuré avec succès.")

## Étape 2 : Chargement et Exploration des Données (SciREX)
Objectif : Télécharger (si nécessaire) et charger le dataset SciREX.

In [2]:
import urllib.request
import tarfile

DATA_DIR = "release_data"
DATA_URL = "https://github.com/allenai/SciREX/raw/master/scirex_dataset/release_data.tar.gz"
TAR_FILE = "release_data.tar.gz"

def download_and_extract_data():
    if not os.path.exists(DATA_DIR):
        print("Dossier de données non trouvé. Téléchargement en cours...")
        try:
            urllib.request.urlretrieve(DATA_URL, TAR_FILE)
            print("Téléchargement terminé. Extraction...")
            with tarfile.open(TAR_FILE, "r:gz") as tar:
                tar.extractall()
            print("Extraction terminée.")
        except Exception as e:
            print(f"Erreur lors du téléchargement/extraction : {e}")
    else:
        print("Les données sont déjà présentes.")

download_and_extract_data()

FILES = {
    "train": os.path.join(DATA_DIR, "train.jsonl"),
    "dev": os.path.join(DATA_DIR, "dev.jsonl"),
    "test": os.path.join(DATA_DIR, "test.jsonl")
}

def load_jsonl(file_path):
    """Charge un fichier JSONL dans une liste de dictionnaires."""
    data = []
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line))
    else:
        print(f"Attention : Fichier {file_path} introuvable.")
    return data

# Chargement des données
print("Chargement des données...")
train_data = load_jsonl(FILES["train"])
dev_data = load_jsonl(FILES["dev"])
test_data = load_jsonl(FILES["test"])

print(f"Nombre de documents Train : {len(train_data)}")
print(f"Nombre de documents Dev   : {len(dev_data)}")
print(f"Nombre de documents Test  : {len(test_data)}")

In [3]:
# Exploration d'un document type
if train_data:
    doc_example = train_data[0]
    print("\nClés disponibles dans un document :")
    print(list(doc_example.keys()))

    print("\nExemple de contenu (champs principaux) :")
    print(f"ID: {doc_example.get('doc_id')}")
    if 'words' in doc_example:
        print(f"Début du texte (50 premiers mots) : {' '.join(doc_example['words'][:50])}...")


## Étape 3 : Nettoyage et Normalisation
Objectif : Nettoyer le texte (lowercase, suppression caractères spéciaux, espaces multiples).

In [4]:
def clean_text(text):
    """
    Nettoie le texte : lowercase, suppression caractères spéciaux, espaces multiples.
    
    Args:
        text (str): Texte à nettoyer
    
    Returns:
        str: Texte nettoyé
    """
    if not isinstance(text, str):
        return ""
    
    # 1. Lowercase
    text = text.lower()
    
    # 2. Suppression des caractères spéciaux (garde lettres, chiffres et espaces)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    
    # 3. Suppression des espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

print("Fonction clean_text() définie.")

In [5]:
# Test sur l'exemple
if train_data and 'words' in doc_example:
    raw_text = ' '.join(doc_example['words'])
    cleaned_text = clean_text(raw_text)
    
    print("Texte original (200 premiers caractères) :")
    print(raw_text[:200])
    print("\nTexte nettoyé (200 premiers caractères) :")
    print(cleaned_text[:200])
    print(f"\nLongueur originale : {len(raw_text)} caractères")
    print(f"Longueur nettoyée  : {len(cleaned_text)} caractères")

In [6]:
# Application du nettoyage sur tous les documents
print("Application du nettoyage sur tous les documents...")

for doc in train_data:
    if 'words' in doc:
        doc['cleaned_text'] = clean_text(' '.join(doc['words']))

for doc in dev_data:
    if 'words' in doc:
        doc['cleaned_text'] = clean_text(' '.join(doc['words']))

for doc in test_data:
    if 'words' in doc:
        doc['cleaned_text'] = clean_text(' '.join(doc['words']))

print(f"Nettoyage terminé pour {len(train_data)} docs train, {len(dev_data)} docs dev, {len(test_data)} docs test.")
print(f"\nExemple de texte nettoyé (doc 0) : {train_data[0]['cleaned_text'][:150]}...")

## Étape 4 : Tokenization, POS Tagging et Lemmatization
Objectif : Tokeniser, identifier les parties du discours (POS) et lemmatiser les textes.

In [7]:
def tokenize_and_pos(text):
    """
    Tokenise le texte et effectue le POS tagging avec NLTK.
    
    Args:
        text (str): Texte à tokeniser
    
    Returns:
        list: Liste de tuples (token, pos_tag)
    """
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return pos_tags

def lemmatize_text(text):
    """
    Lemmatise le texte avec spaCy.
    
    Args:
        text (str): Texte à lemmatiser
    
    Returns:
        str: Texte lemmatisé
    """
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas)

print("Fonctions tokenize_and_pos() et lemmatize_text() définies.")

In [8]:
# Test sur l'exemple
if train_data and 'cleaned_text' in train_data[0]:
    sample_text = train_data[0]['cleaned_text'][:500]  # Premier 500 caractères
    
    print("Texte nettoyé (extrait) :")
    print(sample_text)
    
    print("\n--- Tokenization + POS Tagging ---")
    pos_tags = tokenize_and_pos(sample_text)
    print(f"Nombre de tokens : {len(pos_tags)}")
    print(f"Premiers 10 tokens avec POS : {pos_tags[:10]}")
    
    print("\n--- Lemmatization ---")
    lemmatized = lemmatize_text(sample_text)
    print(f"Texte lemmatisé (extrait) : {lemmatized[:200]}...")

In [9]:
# Application de la lemmatization sur TOUS les documents (train, dev, test)
def process_dataset_lemmatization(dataset, name):
    print(f"Application de la lemmatization sur {len(dataset)} documents {name}...")
    for i, doc in enumerate(dataset):
        if 'cleaned_text' in doc:
            doc['lemmatized_text'] = lemmatize_text(doc['cleaned_text'])
        if (i + 1) % 50 == 0:
            print(f"  Traité {i + 1}/{len(dataset)} documents...")
    print(f"Lemmatization terminée pour {name}.\n")

print("Cela peut prendre quelques minutes...\n")
process_dataset_lemmatization(train_data, "TRAIN")
process_dataset_lemmatization(dev_data, "DEV")
process_dataset_lemmatization(test_data, "TEST")

print(f"Exemple de texte lemmatisé (doc 0 train) : {train_data[0]['lemmatized_text'][:150]}...")

## Étape 5 : Représentation Vectorielle TF-IDF
Objectif : Créer une représentation vectorielle des textes avec TF-IDF sur les textes lemmatisés.

In [10]:
# Préparation des textes pour TF-IDF (utilisation des textes LEMMATISÉS)
train_texts = [doc['lemmatized_text'] for doc in train_data if 'lemmatized_text' in doc]
dev_texts = [doc['lemmatized_text'] for doc in dev_data if 'lemmatized_text' in doc]
test_texts = [doc['lemmatized_text'] for doc in test_data if 'lemmatized_text' in doc]

# Création du vectoriseur TF-IDF
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,  # Limite à 5000 features les plus importantes
    min_df=2,           # Ignore les termes qui apparaissent dans moins de 2 documents
    max_df=0.8,         # Ignore les termes qui apparaissent dans plus de 80% des documents
    ngram_range=(1, 2)  # Unigrammes et bigrammes
)

# Calcul de la matrice TF-IDF sur TRAIN (fit_transform)
print("Calcul de la matrice TF-IDF sur TRAIN (fit_transform)...")
tfidf_matrix = tfidf_vectorizer.fit_transform(train_texts)

# Transformation des ensembles DEV et TEST (transform uniquement)
print("Transformation des ensembles DEV et TEST (transform)...")
tfidf_matrix_dev = tfidf_vectorizer.transform(dev_texts)
tfidf_matrix_test = tfidf_vectorizer.transform(test_texts)

print(f"\nMatrices TF-IDF créées :")
print(f"  TRAIN : {tfidf_matrix.shape} (Densité : {tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]):.4f})")
print(f"  DEV   : {tfidf_matrix_dev.shape} (Densité : {tfidf_matrix_dev.nnz / (tfidf_matrix_dev.shape[0] * tfidf_matrix_dev.shape[1]):.4f})")
print(f"  TEST  : {tfidf_matrix_test.shape} (Densité : {tfidf_matrix_test.nnz / (tfidf_matrix_test.shape[0] * tfidf_matrix_test.shape[1]):.4f})")

In [11]:
# Affichage des top features pour le premier document
feature_names = tfidf_vectorizer.get_feature_names_out()
doc_0_vector = tfidf_matrix[0].toarray()[0]
top_indices = doc_0_vector.argsort()[-10:][::-1]

print("Top 10 features TF-IDF pour le document 0 :")
for idx in top_indices:
    print(f"  {feature_names[idx]}: {doc_0_vector[idx]:.4f}")

## Étape 6 : Export et Sauvegarde des Résultats
Objectif : Sauvegarder tous les résultats pour la Partie B.

In [12]:
# Création du dossier de sortie
OUTPUT_DIR = "preprocessed_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Dossier de sortie créé : {OUTPUT_DIR}/")

### 6.1 - Export des textes prétraités (CSV)

In [13]:
def export_to_csv(dataset, filename):
    df = pd.DataFrame([
        {
            'doc_id': doc.get('doc_id', f'doc_{i}'),
            'raw_text': ' '.join(doc.get('words', [])),
            'cleaned_text': doc.get('cleaned_text', ''),
            'lemmatized_text': doc.get('lemmatized_text', '')
        }
        for i, doc in enumerate(dataset)
    ])
    df.to_csv(os.path.join(OUTPUT_DIR, filename), index=False, encoding='utf-8')
    print(f"✅ Fichier sauvegardé : {OUTPUT_DIR}/{filename}")
    print(f"   Colonnes : {list(df.columns)}")
    print(f"   Nombre de lignes : {len(df)}")

export_to_csv(train_data, 'train_preprocessed.csv')
export_to_csv(dev_data, 'dev_preprocessed.csv')
export_to_csv(test_data, 'test_preprocessed.csv')

### 6.2 - Export de la matrice TF-IDF

In [14]:
# Sauvegarde des matrices TF-IDF (format sparse)
save_npz(os.path.join(OUTPUT_DIR, 'tfidf_matrix.npz'), tfidf_matrix)
save_npz(os.path.join(OUTPUT_DIR, 'tfidf_matrix_dev.npz'), tfidf_matrix_dev)
save_npz(os.path.join(OUTPUT_DIR, 'tfidf_matrix_test.npz'), tfidf_matrix_test)

print(f"✅ Matrice TF-IDF TRAIN sauvegardée : {OUTPUT_DIR}/tfidf_matrix.npz")
print(f"✅ Matrice TF-IDF DEV sauvegardée   : {OUTPUT_DIR}/tfidf_matrix_dev.npz")
print(f"✅ Matrice TF-IDF TEST sauvegardée  : {OUTPUT_DIR}/tfidf_matrix_test.npz")

# Sauvegarde du vectoriseur (pour réutilisation)
with open(os.path.join(OUTPUT_DIR, 'tfidf_vectorizer.pkl'), 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)
print(f"✅ Vectoriseur TF-IDF sauvegardé : {OUTPUT_DIR}/tfidf_vectorizer.pkl")

# Sauvegarde des noms de features
np.save(os.path.join(OUTPUT_DIR, 'tfidf_feature_names.npy'), feature_names)
print(f"✅ Noms des features sauvegardés : {OUTPUT_DIR}/tfidf_feature_names.npy")

### 6.3 - Export du dictionnaire de correspondance

In [15]:
# Création d'un dictionnaire de correspondance complet
correspondence_dict = {
    'metadata': {
        'n_documents_train': len(train_data),
        'n_documents_dev': len(dev_data),
        'n_documents_test': len(test_data),
        'tfidf_shape_train': tfidf_matrix.shape,
        'tfidf_shape_dev': tfidf_matrix_dev.shape,
        'tfidf_shape_test': tfidf_matrix_test.shape,
        'n_features': len(feature_names),
        'preprocessing_steps': [
            '1. Lowercase',
            '2. Suppression caractères spéciaux',
            '3. Normalisation espaces',
            '4. Tokenization (NLTK)',
            '5. POS Tagging (NLTK)',
            '6. Lemmatization (spaCy)',
            '7. TF-IDF sur textes lemmatisés (max_features=5000, ngram_range=(1,2))'
        ]
    },
    'documents_train_sample': [
        {
            'doc_id': doc.get('doc_id', f'doc_{i}'),
            'raw_text_preview': ' '.join(doc.get('words', []))[:200],
            'cleaned_text_preview': doc.get('cleaned_text', '')[:200],
            'lemmatized_text_preview': doc.get('lemmatized_text', '')[:200],
            'tfidf_vector_index': i
        }
        for i, doc in enumerate(train_data[:5])
    ]
}

# Sauvegarde en JSON
with open(os.path.join(OUTPUT_DIR, 'correspondence_dict.json'), 'w', encoding='utf-8') as f:
    json.dump(correspondence_dict, f, indent=2, ensure_ascii=False)

print(f"✅ Dictionnaire de correspondance sauvegardé : {OUTPUT_DIR}/correspondence_dict.json")

### 6.4 - Résumé des fichiers exportés

In [16]:
import os

print("\n" + "="*60)
print("RÉSUMÉ DES FICHIERS EXPORTÉS")
print("="*60)

for filename in sorted(os.listdir(OUTPUT_DIR)):
    filepath = os.path.join(OUTPUT_DIR, filename)
    if os.path.isfile(filepath):
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"📄 {filename:35s} ({size_mb:.2f} MB)")

print("="*60)

## Résumé Technique - Partie A

### Pipeline de Preprocessing Complet

1. **Nettoyage** :
   - Conversion en lowercase
   - Suppression des caractères spéciaux (garde uniquement lettres, chiffres, espaces)
   - Normalisation des espaces multiples

2. **Tokenization** :
   - Tokenization avec NLTK (`word_tokenize`)

3. **POS Tagging** :
   - POS tagging avec NLTK (`pos_tag`)

4. **Lemmatization** :
   - Lemmatization avec spaCy (`en_core_web_sm`)
   - **Appliqué sur TOUS les documents (Train, Dev, Test)**

5. **Représentation TF-IDF** :
   - Vectorisation avec scikit-learn `TfidfVectorizer`
   - **Apprentissage (Fit)** sur le TRAIN set uniquement
   - **Transformation** appliquée sur TRAIN, DEV et TEST
   - Paramètres : `max_features=5000`, `min_df=2`, `max_df=0.8`, `ngram_range=(1,2)`
   - Matrice résultante : N documents × 5000 features

---

### Fichiers Exportés pour la Partie B

| Fichier | Description | Format |
|---------|-------------|--------|
| `train_preprocessed.csv` | Textes TRAIN prétraités | CSV |
| `dev_preprocessed.csv` | Textes DEV prétraités | CSV |
| `test_preprocessed.csv` | Textes TEST prétraités | CSV |
| `tfidf_matrix.npz` | Matrice TF-IDF TRAIN (sparse) | NumPy compressed |
| `tfidf_matrix_dev.npz` | Matrice TF-IDF DEV (sparse) | NumPy compressed |
| `tfidf_matrix_test.npz` | Matrice TF-IDF TEST (sparse) | NumPy compressed |
| `tfidf_vectorizer.pkl` | Vectoriseur TF-IDF entraîné | Pickle |
| `tfidf_feature_names.npy` | Noms des 5000 features TF-IDF | NumPy |
| `correspondence_dict.json` | Métadonnées et correspondances | JSON |

---

### Comment Charger les Données (Partie B)

```python
import pandas as pd
import numpy as np
import pickle
from scipy.sparse import load_npz

# Charger les textes prétraités
df_train = pd.read_csv('preprocessed_data/train_preprocessed.csv')
df_dev = pd.read_csv('preprocessed_data/dev_preprocessed.csv')

# Charger les matrices TF-IDF
tfidf_train = load_npz('preprocessed_data/tfidf_matrix.npz')
tfidf_dev = load_npz('preprocessed_data/tfidf_matrix_dev.npz')

# Charger le vectoriseur (pour transformer de nouveaux textes)
with open('preprocessed_data/tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# Charger les noms de features
feature_names = np.load('preprocessed_data/tfidf_feature_names.npy')
```

---

### Statistiques Finales

- **Documents train** : 306
- **Documents dev** : 66
- **Documents test** : 66
- **Features TF-IDF** : 5000
- **Pipeline** : Nettoyage → Tokenization → POS → Lemmatization → TF-IDF

---

**Note** : Le pipeline complet garantit que TF-IDF est calculé sur des textes entièrement prétraités (lemmatisés), ce qui améliore la qualité des représentations vectorielles.